# Admission Prediction with PySpark

![](https://miro.medium.com/max/500/1*5C4UQznqEiN3D6Xutlgwlg.png)

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bb0f869268016bc3c37543f1af17185bb05d1c926e41434a712e699f664bb7db
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName("ml_project").getOrCreate()

In [3]:
spark

In [31]:
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv', header=True, inferSchema=True)

In [32]:
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [33]:
shape = (df.count(), len(df.columns))

print(shape)

(500, 9)


In [8]:
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [9]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [10]:
df= df.drop('Serial No')

In [11]:
df.show(2)

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 2 rows



In [12]:
print(df[df['GRE Score'].isNull()].count())

0


In [13]:
for col in df.columns:
  print(col + ':' , df[df[col].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


### Correlation Analysis & Feature Selection

In [14]:
print(df.stat.corr('GRE Score', 'Chance of Admit'))

0.8103506354632601


In [15]:
for col in df.columns:
  print(f"{col} is {round(df.stat.corr(col, 'Chance of Admit'),3)} correlated with the target variable Chance of Admit")

GRE Score is 0.81 correlated with the target variable Chance of Admit
TOEFL Score is 0.792 correlated with the target variable Chance of Admit
University Rating is 0.69 correlated with the target variable Chance of Admit
SOP is 0.684 correlated with the target variable Chance of Admit
LOR is 0.645 correlated with the target variable Chance of Admit
CGPA is 0.882 correlated with the target variable Chance of Admit
Research is 0.546 correlated with the target variable Chance of Admit
Chance of Admit is 1.0 correlated with the target variable Chance of Admit


In [16]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score', 'CGPA'],outputCol='features' )

In [17]:
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

###  Linear Regression Model

In [18]:
from pyspark.ml.regression import LinearRegression

final_data = output_data.select('features', 'Chance of Admit')

In [19]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [20]:
train, test = final_data.randomSplit([0.7,0.3])


In [21]:
models = LinearRegression(featuresCol= 'features', labelCol='Chance of Admit')

model= models.fit(train)


In [22]:
print('Coefficients:', model.coefficients)

print('Intercept:', model.intercept)

Coefficients: [0.0021479207347598023,0.002922830728822597,0.1441916791869408]
Intercept: -1.5088795827896033


In [23]:
summary = model.summary

In [24]:
print('RMSE :', summary.rootMeanSquaredError)

print('r2 :', summary.r2)

RMSE : 0.06486527060867608
r2 : 0.7882181698326369


###  Evaluate & Save the Model

In [25]:
predictions= model.transform(test)

In [26]:
predictions.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [293.0,97.0,7.8]|           0.64| 0.5286708708489491|
| [294.0,93.0,7.36]|           0.46|0.45568312982616455|
| [295.0,96.0,7.34]|           0.47|0.46371570916365323|
| [295.0,99.0,7.65]|           0.57| 0.5171836218980725|
| [296.0,95.0,7.54]|           0.44|0.49177913500697845|
| [296.0,99.0,8.03]|           0.61| 0.5741243807238698|
| [297.0,96.0,7.43]|           0.34| 0.4809888017599975|
| [297.0,98.0,7.67]|           0.59| 0.5214404662225085|
| [297.0,99.0,7.81]|           0.54|  0.544550132037503|
| [297.0,100.0,7.9]|           0.52|   0.56045021389315|
|  [298.0,98.0,7.5]|           0.44|0.49907580149548836|
|  [298.0,99.0,7.6]|           0.46| 0.5164178001430051|
|[298.0,100.0,7.95]|           0.58|  0.569807718587257|
|[298.0,101.0,7.86]|           0.54|  0.559753298189255|
|[298.0,105.0,8.54]|           

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit', metricName='r2')

print('r2 score:', evaluator.evaluate(predictions))

r2 score: 0.8359574010929498


In [28]:
model.save('model')

In [29]:
from pyspark.ml.regression import LinearRegressionModel

model_new= LinearRegressionModel.load('model')

In [30]:
model_new

LinearRegressionModel: uid=LinearRegression_0aa7e48dcbdc, numFeatures=3